In [1]:
# I. Import des bibliothèques
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# II. Lecture des données
df = pd.read_csv('tableau_pour_apprentissage.csv', sep=';')
df2 = pd.read_csv('tableau_a_remplir.csv', sep=';')

# III.1 Filtrage sur 'CHE' sans échantillonnage
df_masque = df[df['cespece'] == 'CHE']

# III.2 Ajout d'une colonne taille moyenne
group_cols = ['Secteur', 'NumStation', 'Month', 'Year']
df_masque['Taille moyenne'] = df_masque.groupby(group_cols)['taille (cm)'].transform('mean')
df2['Taille moyenne'] = df2.groupby(group_cols)['taille (cm)'].transform('mean')

# IV. Données quantitatives
df_quanti = df_masque[['Secteur', 'NumStation', 'Month', 'Year', 'Taille moyenne']]

# V. Centre-réduction
scaler = StandardScaler()
scaler.fit(df_quanti) 
df_scaled = pd.DataFrame(scaler.transform(df_quanti), columns=['Secteur','NumStation','Month','Year','Taille moyenne'])

# VI. Features et label
df_features = df_scaled.dropna()
df_label = df_masque['taille (cm)']

# VII. Grid Search pour les meilleurs paramètres
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 3, 5],
    'max_features': ['sqrt', 'log2', None]
}

grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(df_features, df_label)
print("Meilleurs paramètres :", grid_search.best_params_)
print("Meilleur score R² :", grid_search.best_score_)

# VIII.1 Entraînement du modèle final
best_rf = grid_search.best_estimator_
best_rf.fit(df_features, df_label)  

# VIII.2 Évaluer le modèle sur les données d'entraînement
pred_train_y = best_rf.predict(df_features)
rmse_train = np.sqrt(mean_squared_error(df_label, pred_train_y))
print("RMSE sur les données d'entraînement :", rmse_train)

# IX. Préparation des données à remplir
df2_quanti = df2[['Secteur', 'NumStation', 'Month', 'Year', 'Taille moyenne']]
df2_scaled = pd.DataFrame(scaler.transform(df2_quanti), columns=df2_quanti.columns)

# X.1 Prédictions sur df2 avec le modèle final
df2['taille (cm)'] = best_rf.predict(df2_scaled)

# X.2 Ajout de bruit aléatoire aux prédictions finales pour simuler une variabilité
df2['taille (cm)'] += np.random.normal(0, 0.2, size=len(df2))
df2['Taille moyenne'] = df2.groupby(group_cols)['taille (cm)'].transform('mean')

# XI. Remplissage colonne juvénile
df2['Juvenile'] = df2['taille (cm)'].apply(lambda x: 'oui' if x < 6.5 else 'non')

# XII. Sauvegarde
df2.to_csv("fichier_final.csv", index=False)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


/tmp/ipykernel_2536/2567645917.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_masque['Taille moyenne'] = df_masque.groupby(group_cols)['taille (cm)'].transform('mean')


Meilleurs paramètres : {'max_depth': 5, 'max_features': None, 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 200}
Meilleur score R² : 0.18082535563085503
RMSE sur les données d'entraînement : 7.2439259487831835
